In [1]:
import numpy as npt
import pandas as pd
import matplotlib.pyplot as plt
# from gym import wrappers
# from gym import spaces
import math
import random
import re
import os
import sys
import seaborn as sns

In [2]:
import os

def functionfilenameandpath(path='./', extension='.em1.pcap'):
    filepaths = []
    filenames = []

    for root, _, files_list in os.walk(path):
        for file_name in files_list:
            if file_name.endswith(extension):
                file_name_path = os.path.join(root, file_name)
                filenames.append(file_name)
                filepaths.append(file_name_path)

    return filenames, filepaths
folderpath="./client1_data"
filenames,filepaths=functionfilenameandpath(folderpath,'em1.pcap')

filepaths.sort()
filenames.sort()
print(filenames)

['10_10_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '1_1_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '2_2_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '3_3_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '4_4_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '5_5_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '6_6_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '7_7_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '8_8_l4s_8Mbps_10ms_newreno_src1.em1.pcap', '9_9_l4s_8Mbps_10ms_newreno_src1.em1.pcap']


In [3]:
pcap_file = filepaths[0]

In [4]:
import re

def contains_1_or_2(input_string):
    if '2' in input_string:
        return 2
    elif '1' in input_string:
        return 1
    else:
        return 0

# Example usage:
test_string = "abc123xyz"
result = contains_1_or_2(test_string)
print("Result:", result)


Result: 2


In [5]:
# Read packets from the pcap file
pcap_file = './client1_data/fq_pie_1Mbps_20ms_ecn_cubic_src1.em1.pcap'

In [6]:
# Read packets from the pcap file
pcap_file = './client2_data/fq_pie_1Mbps_20ms_ecn_dctcp_src2.em3.pcap'

In [7]:
import dpkt
import socket


# Define the maximum port and window size
def get_tcp_window_scale_option(tcp_options):
    for opt in dpkt.tcp.parse_opts(tcp_options):
        if opt[0] == dpkt.tcp.TCP_OPT_WSCALE:
            scale_factor = int.from_bytes(opt[1], byteorder='big')
            return 2 ** scale_factor
    return 1  # Default scale if option is not present


def analysis_pcap_tcp(pcap_file):
    with open(pcap_file, 'rb') as f:
        pcap = dpkt.pcap.Reader(f)
        tcp_flows = {}
        sender_ip = None

        for timestamp, buf in pcap:
            try:
                # Parse the packet
                eth = dpkt.ethernet.Ethernet(buf)
                # Ensure the packet is an IP packet
                if not isinstance(eth.data, dpkt.ip.IP):
                    continue
                ip = eth.data
                # Ensure the packet is a TCP packet
                if not isinstance(ip.data, dpkt.tcp.TCP):
                    continue
                tcp = ip.data

                # Extract relevant information from the packet
                src_ip = socket.inet_ntoa(ip.src)
                dst_ip = socket.inet_ntoa(ip.dst)

                # Determine if this is the first packet in the conversation to set the sender
                if sender_ip is None and (tcp.flags & dpkt.tcp.TH_SYN):
                    sender_ip = src_ip  # The sender is the one who sends the first SYN
                if sender_ip != dst_ip and sender_ip != src_ip and (tcp.flags & dpkt.tcp.TH_SYN):
                    sender_ip = src_ip  # The sender is the one who sends the first SYN
                # If this packet is not from the sender, ignore it
                if src_ip != sender_ip:
                    continue

                # Extract the source and destination ports
                src_port = tcp.sport
                dst_port = tcp.dport

                # Sort IP addresses and ports to identify flows uniquely irrespective of direction.
                flow_id = (src_ip, src_port, dst_ip, dst_port)

                # Ensure flow is initialized for any TCP packet, adjusting the logic accordingly.
                if flow_id not in tcp_flows:
                    tcp_flows[flow_id] = {
                        'start_time': None, 'end_time': timestamp, 'data_bytes': 0,
                        'transactions': [], 'handshake_complete': False, 'packet_count': 0, 
                        'flow_tuple': flow_id, 'window_scale': 1, 'syn': False, 'fin': False,
                        'initial_rtt': None, 'rtt_estimation': None, 'cwnd_packets': [], 
                        'last_packet_time': None, 'ack_freq': {}, 'seq_numbers': {},   
                        'triple_dup_acks': 0, 'timeouts': 0, 'retransmissions': -1, 'seen_seqs': {},
                    }

                # Get the flow object
                flow = tcp_flows[flow_id]
                flow['end_time'] = timestamp

                # Update the flow information
                if flow['start_time'] is None and tcp.flags & dpkt.tcp.TH_SYN:
                    flow['start_time'] = timestamp  # Set the start time of the flow
                    flow['window_scale'] = get_tcp_window_scale_option(tcp.opts) # Get the window scale option
                if flow['start_time'] is not None:
                    flow['data_bytes'] += len(tcp)
                    flow['packet_count'] += 1
                    
                # Check for handshake completion
                if tcp.flags & dpkt.tcp.TH_SYN and not tcp.flags & dpkt.tcp.TH_ACK:
                    flow['syn'] = True
                if tcp.flags & dpkt.tcp.TH_ACK and flow['syn'] and not flow['handshake_complete']:
                    flow['handshake_complete'] = True

                # Check for flow termination
                if tcp.flags & dpkt.tcp.TH_FIN | dpkt.tcp.TH_RST:
                    flow['fin'] = True
                
                
                # Check for data packets
                if flow['handshake_complete'] and len(tcp.data) > 0:
                    # Add the transaction to the flow if one of first 2
                    if len(flow['transactions']) < 2:
                        flow['transactions'].append((tcp.seq, tcp.ack, tcp.win * flow['window_scale']))

                # Adjusting logic to capture initial RTT and counting packets per RTT
                if tcp.flags & dpkt.tcp.TH_SYN and not flow['initial_rtt']:
                    flow['last_packet_time'] = timestamp  # Mark the time of the SYN packet
                elif tcp.flags & dpkt.tcp.TH_ACK and flow['syn'] and not flow['initial_rtt']:
                    # Assuming this ACK is part of the handshake completion
                    flow['initial_rtt'] = timestamp - flow['last_packet_time']  # Initial RTT estimation
                    flow['rtt_estimation'] = flow['initial_rtt']  # Storing initial RTT as the estimate
                    flow['last_packet_time'] = timestamp  # Resetting for packet counting

                # Logic to count packets per estimated RTT
                if flow['initial_rtt']:
                    time_since_last_packet = timestamp - flow['last_packet_time']
                    if time_since_last_packet < flow['rtt_estimation']:
                        # Still within the same RTT, count the packet
                        if len(flow['cwnd_packets']) == 0:
                            flow['cwnd_packets'].append(1)  # Starting the first count
                        else:
                            flow['cwnd_packets'][-1] += 1  # Increment current RTT's packet count 
                            pass
                    else:
                        # New RTT period, reset packet count
                        if len(flow['cwnd_packets']) < 4:  # Only if we need more CWND sizes
                            flow['cwnd_packets'].append(1)  # Start counting for a new CWND size
                        flow['last_packet_time'] = timestamp  # Resetting for next RTT counting

                if (tcp.flags & dpkt.tcp.TH_ACK) and not (tcp.flags & dpkt.tcp.TH_SYN):
                    ack = tcp.ack
                    # Track ACKs for triple duplicate detection
                    if ack not in flow['ack_freq']:
                        flow['ack_freq'][ack] = 1
                    else:
                        flow['ack_freq'][ack] += 1
                        current_ack_freq = flow['ack_freq'][ack]
                        if current_ack_freq == 4:  # Triple duplicate ACK detected
                            flow['triple_dup_acks'] += 1

                # Track sequence numbers to identify retransmissions
                if tcp.seq not in flow['seq_numbers']:
                    flow['seq_numbers'][tcp.seq] = timestamp
                else:
                    # Check if the packet is a retransmission
                    if timestamp - flow['seq_numbers'][tcp.seq] > 2 * flow['rtt_estimation']: # Timeout threshold
                        flow['timeouts'] += 1

                # Track sequence numbers to detect retransmissions
                seq = tcp.seq
                if flow['handshake_complete'] and seq in flow['seen_seqs'] and flow['seen_seqs'][seq] is True:
                    # If the sequence number has been seen before, it's a retransmission
                    flow['retransmissions'] += 1
                    flow['seen_seqs'][seq] = False
                elif flow['handshake_complete']:
                    # Otherwise, mark the sequence number as seen
                    flow['seen_seqs'][seq] = True

            except Exception as e:
                # Print the error and continue to the next packet
                print(f"Error processing packet: {e}")

    # Output results
    print(f"\nNumber of TCP flows initiated from the sender: {len(tcp_flows)}\n")
    flow_count = 1
    for flow_id, data in tcp_flows.items():
        print(f"Flow {flow_count}:")
        flow_count += 1
        print(flow)
        print(f"  Source IP: {data['flow_tuple'][0]}, Source Port: {data['flow_tuple'][1]} -> Destination IP: {data['flow_tuple'][2]}, Destination Port: {data['flow_tuple'][3]}")
        for i, transaction in enumerate(data['transactions'], 1):
            seq_num, ack_num, rec_win_size = transaction
            print(f"  Transaction {i}: Seq Num: {seq_num}, Ack Num: {ack_num}, Rec Win Size: {rec_win_size}")
        if 'end_time' in data and data['end_time'] is not None:
            duration = data['end_time'] - data['start_time']
            throughput = data['data_bytes'] / duration if duration > 0 else 0
            print(f"  Sender Throughput: {throughput} bytes/sec")
            print(f"  == ({data['data_bytes']} bytes sent in {data['end_time'] - data['start_time']} seconds)")
            print(f"  First 3 Congestion Window Sizes: {data['cwnd_packets'][0:3]}")
            print(f"  Retransmissions due to Triple Duplicate ACKs: {data['retransmissions'] - data['timeouts']}")
            print(f"  Retransmissions due to Timeouts: {data['timeouts']}")
        else:
            print("  Incomplete flow.")
        print()


In [9]:
analysis_pcap_tcp('./client1_data/1_1_l4s_8Mbps_10ms_newreno_src1.em1.pcap')


Number of TCP flows initiated from the sender: 6

Flow 1:
{'start_time': 1727695337.469256, 'end_time': 1727695398.728536, 'data_bytes': 1091, 'transactions': [(3013720308, 3345817137, 65728), (3013720345, 3345817138, 65728)], 'handshake_complete': True, 'packet_count': 14, 'flow_tuple': ('172.16.3.2', 22720, '172.16.1.2', 5102), 'window_scale': 64, 'syn': True, 'fin': True, 'initial_rtt': 0.10776495933532715, 'rtt_estimation': 0.10776495933532715, 'cwnd_packets': [4, 1, 1, 5], 'last_packet_time': 1727695398.639584, 'ack_freq': {3345817137: 2, 3345817138: 2, 3345817139: 1, 3345817141: 2, 3345817142: 2, 3345817625: 1, 3345817626: 2, 3345817627: 1}, 'seq_numbers': {3013720307: 1727695337.469256, 3013720308: 1727695337.577021, 3013720345: 1727695337.683022, 3013720349: 1727695337.683229, 3013720475: 1727695337.826368, 3013720476: 1727695398.525592, 3013720480: 1727695398.525704, 3013720942: 1727695398.639584, 3013720943: 1727695398.640225, 3013720944: 1727695398.728536}, 'triple_dup_acks

In [ ]:
analysis_pcap_tcp('./client2_data/l4s_10Mbps_20ms_ecn_dctcp_src2.em3.pcap')


Number of TCP flows initiated from the sender: 2

Flow 1:
{'start_time': 1723549325.949871, 'end_time': 1723549387.018703, 'data_bytes': 958, 'transactions': [(138217126, 2708074072, 65728), (138217163, 2708074073, 65728)], 'handshake_complete': True, 'packet_count': 14, 'flow_tuple': ('172.16.5.2', 27549, '172.16.1.2', 5102), 'window_scale': 64, 'syn': True, 'fin': True, 'initial_rtt': 0.12454795837402344, 'rtt_estimation': 0.12454795837402344, 'cwnd_packets': [4, 1, 1, 5], 'last_packet_time': 1723549387.018703, 'ack_freq': {2708074072: 2, 2708074073: 2, 2708074074: 1, 2708074076: 2, 2708074077: 2, 2708074391: 1, 2708074392: 2, 2708074393: 1}, 'seq_numbers': {138217125: 1723549325.949871, 138217126: 1723549326.074419, 138217163: 1723549326.189522, 138217167: 1723549326.189654, 138217293: 1723549326.404597, 138217294: 1723549386.785126, 138217298: 1723549386.78573, 138217627: 1723549386.892733, 138217628: 1723549386.8947}, 'triple_dup_acks': 0, 'timeouts': 1, 'retransmissions': 3, 'se

In [ ]:
analysis_pcap_tcp('./client1_data/l4s_1Mbps_20ms_ecn_cubic_src1.em1.pcap')


Number of TCP flows initiated from the sender: 2

Flow 1:
{'start_time': 1723549152.245089, 'end_time': 1723549213.305177, 'data_bytes': 987, 'transactions': [(2491598905, 1195371186, 65728), (2491598942, 1195371187, 65728)], 'handshake_complete': True, 'packet_count': 15, 'flow_tuple': ('172.16.3.2', 41508, '172.16.1.2', 5101), 'window_scale': 64, 'syn': True, 'fin': True, 'initial_rtt': 0.12132787704467773, 'rtt_estimation': 0.12132787704467773, 'cwnd_packets': [2, 2, 1, 5], 'last_packet_time': 1723549213.305041, 'ack_freq': {1195371186: 2, 1195371187: 2, 1195371188: 1, 1195371190: 2, 1195371191: 2, 1195371523: 1, 1195371524: 2, 1195371525: 2}, 'seq_numbers': {2491598904: 1723549152.245089, 2491598905: 1723549152.366417, 2491598942: 1723549152.488229, 2491598946: 1723549152.488795, 2491599072: 1723549152.643718, 2491599073: 1723549213.007877, 2491599077: 1723549213.008066, 2491599403: 1723549213.159855, 2491599404: 1723549213.161591, 2491599405: 1723549213.305177}, 'triple_dup_acks'

In [ ]:
analysis_pcap_tcp('./client2_data/l4s_1Mbps_20ms_ecn_dctcp_src2.em3.pcap')


Number of TCP flows initiated from the sender: 2

Flow 1:
{'start_time': 1723549151.613019, 'end_time': 1723549212.895793, 'data_bytes': 938, 'transactions': [(3158046441, 1013220811, 65728), (3158046478, 1013220812, 65728)], 'handshake_complete': True, 'packet_count': 14, 'flow_tuple': ('172.16.5.2', 56517, '172.16.1.2', 5102), 'window_scale': 64, 'syn': True, 'fin': True, 'initial_rtt': 0.11503911018371582, 'rtt_estimation': 0.11503911018371582, 'cwnd_packets': [2, 2, 1, 4], 'last_packet_time': 1723549212.895793, 'ack_freq': {1013220811: 2, 1013220812: 2, 1013220813: 1, 1013220815: 2, 1013220816: 2, 1013221131: 1, 1013221132: 2, 1013221133: 1}, 'seq_numbers': {3158046440: 1723549151.613019, 3158046441: 1723549151.728058, 3158046478: 1723549151.848961, 3158046482: 1723549151.849399, 3158046608: 1723549152.090098, 3158046609: 1723549212.439094, 3158046613: 1723549212.439248, 3158046922: 1723549212.749002, 3158046923: 1723549212.74964}, 'triple_dup_acks': 0, 'timeouts': 1, 'retransmiss